# Problem 14: Head-Dependent Boundary Conditions

### Part e)
_Model the system described above using a two-layer model. The top layer will be the same as in parts ad, except a third type boundary will not be explicitly included. Instead, the bottom layer will represent the third type boundary condition. The bottom layer will be inactive except along column 4, which will be constant head of 0.0 ft. The bottom layer will be confined and have a transmissivity of 100 ft2/d. Calculate a VCONT between layers 1 and 2 to give a conductance of 200 ft2/d. Run the model and note results as you did in parts a-d._

In [ ]:
#print discharge values (Table 14.3)
fname = os.path.join(model_ws, 'p14_pc.cbc')
cbb = flopy.utils.CellBudgetFile(fname)
frf = cbb.get_data(text='FLOW RIGHT FACE')[0]
fff = cbb.get_data(text='FLOW FRONT FACE')[0]
# = cbb.get_data(text='CONSTANT HEAD')[0] #create an array/for loop to get out first value of every step
#cbb.list_records()

#print(fff)
#print()

### Table 14.2. Hydraulic head at node (1,4) for part e representing the third type boundary condition

Check your results to make sure they match with the table from the book

| Time Step | Elapsed Time (days) | Constant head (part e) |
|-----------|---------------------|------------------------|
| 1         | 1.955               | -1702.071              |
| 2         | 4.301               | -1507.91               |
| 3         | 7.117               | -1343.224              |
| 4         | 10.495              | -1185.109              |
| 5         | 14.549              | -1025.712              |
| 6         | 19.414              | -862.6                 |
| 7         | 25.252              | -696.128               |
| 8         | 32.258              | -528.506               |
| 9         | 40.655              | -363.187               |
| 10        | 50.753              | -204.534               |
| 11        | 62.858              | -57.278                |
| 12        | 77.385              | 74.008                 |
| 13        | 94.817              | 186.017                |
| 14        | 115.74              | 276.478                |
| 15        | 140.84              | 345.343                |
| 16        | 170.96              | 394.331                |
| 17        | 207.11              | 426.631                |
| 18        | 250.49              | 446.21                 |
| 19        | 302.54              | 457.025                |
| 20        | 365                 | 462.099                |

### MODEL RESULT
_Table 14.2 shows hydraulic head versus time at node (1,4) for each of the five parts to this problem. Table 14.3 shows discharge versus time at node (1,4). Hydraulic head versus flow is plotted in Figure 14.1._

### Table 14.2. Hydraulic head at node (1,4) for each of the five methods of representing the third type boundary condition

| Time Step | Ellapsed Time (days) | River (part a) | GHB (part b) | Drain (part c) | E-T (part d) | Constant head (part e) |
|-----------|----------------------|----------------|--------------|----------------|--------------|------------------------|
| 1         | 1.955                | 8.51           | 8.51         | 8.51           | 8.51         | 8.51                   |
| 2         | 4.301                | 7.54           | 7.54         | 7.54           | 7.54         | 7.54                   |
| 3         | 70117                | 6.716          | 6.716        | 6.716          | 6.716        | 6.716                  |
| 4         | 10.495               | 5.926          | 5.926        | 5.926          | 5.926        | 5.926                  |
| 5         | 14.549               | 5.129          | 5.129        | 5.129          | 5.129        | 5.129                  |
| 6         | 19.414               | 4.313          | 4.313        | 4.313          | 4.313        | 4.313                  |
| 7         | 25.252               | 3.481          | 3.481        | 3.481          | 3.481        | 3.481                  |
| 8         | 32.258               | 2.643          | 2.643        | 2.643          | 2.643        | 2.643                  |
| 9         | 40.655               | 1.816          | 1.816        | 1.816          | 1.816        | 1.816                  |
| 10        | 50.753               | 1.023          | 1.023        | 1.023          | 1.023        | 1.023                  |
| 11        | 62.858               | 0.286          | 0.286        | 0.286          | 0.286        | 0.286                  |
| 12        | 77.385               | -0.371         | -0.371       | -0.495         | -0.495       | -0.37                  |
| 13        | 94.817               | -0.93          | -0.93        | -1.44          | -1.44        | -0.93                  |
| 14        | 115.74               | -1.382         | -1.382       | -2.54          | -2.54        | -1.382                 |
| 15        | 140.84               | -1.727         | -1.727       | -3.843         | -3.843       | -1.727                 |
| 16        | 170.96               | -1.972         | -1.972       | -5.399         | -5.399       | -1.972                 |
| 17        | 207.11               | -2.175         | -2.133       | -7.264         | -7.264       | -2.133                 |
| 18        | 250.49               | -2.337         | -2.231       | -9.505         | -9.505       | -2.231                 |
| 19        | 302.54               | -2.448         | -2.285       | -12.2          | -12.2        | -2.285                 |
| 20        | 365                  | -2.52          | -2.311       | -15.43         | -15.43       | -2.31

### Table 14.3. Discharge for each of the five methods of representing the third type boundary condition

| Time Step | Elapsed Time (days) | River (part a) | GHB (part b) | Drain (part c) | E-T (part d) | Constant head (part e) |
|-----------|---------------------|----------------|--------------|----------------|--------------|------------------------|
| 1         | 1.955               | -1702.071      | -1702.071    | -1702.071      | -1702.071    | -1702.071              |
| 2         | 4.301               | -1507.91       | -1507.91     | -1507.91       | -1507.91     | -1507.91               |
| 3         | 7.117               | -1343.224      | -1343.224    | -1343.224      | -1343.224    | -1343.224              |
| 4         | 10.495              | -1185.109      | -1185.109    | -1185.109      | -1185.109    | -1185.109              |
| 5         | 14.549              | -1025.712      | -1025.712    | -1025.712      | -1025.712    | -1025.712              |
| 6         | 19.414              | -862.6         | -862.6       | -862.6         | -862.6       | -862.6                 |
| 7         | 25.252              | -696.128       | -696.128     | -696.128       | -696.128     | -696.128               |
| 8         | 32.258              | -528.506       | -528.506     | -528.506       | -528.506     | -528.506               |
| 9         | 40.655              | -363.187       | -363.187     | -363.187       | -363.187     | -363.187               |
| 10        | 50.753              | -204.534       | -204.534     | -204.534       | -204.534     | -204.534               |
| 11        | 62.858              | -57.278        | -57.278      | -57.278        | -57.278      | -57.278                |
| 12        | 77.385              | 74.103         | 74.103       | 0              | 0            | 74.008                 |
| 13        | 94.817              | 186.031        | 186.031      | 0              | 0            | 186.017                |
| 14        | 115.74              | 276.494        | 276.494      | 0              | 0            | 276.478                |
| 15        | 140.84              | 345.353        | 345.353      | 0              | 0            | 345.343                |
| 16        | 170.96              | 394.344        | 394.344      | 0              | 0            | 394.331                |
| 17        | 207.11              | 400            | 426.648      | 0              | 0            | 426.631                |
| 18        | 250.49              | 400            | 446.221      | 0              | 0            | 446.21                 |
| 19        | 302.54              | 400            | 457.037      | 0              | 0            | 457.025                |
| 20        | 365                 | 400            | 462.149      | 0              | 0            | 462.099                |

# DISCUSSION OF RESULTS

_The head-dependent boundary conditions in MODFLOW are all very similar. They allow leakage into or out of the system depending upon the difference in head between the aquifer and some constant head external to the system. The amount of leakage is controlled by a conductance term, which establishes the degree of hydraulic connection between the aquifer and the external source/sink. This exercise demonstrates the equivalency of the boundaries and highlights some of the differences._

_The general head boundary is, as the name implies. a general leakage boundary. Flux is directly proportional to head difference for the entire range of saturated conditions. The drain is essentially a one-way general head boundary. Flow can only be out of the aquifer. Evapotranspiration is posed in terms of rates, but it is equivalent to a drain with an upper limit on flux out of the aquifer. Flow cuts off at a certain depth and can only reach a certain upper threshold. Finally, the river is a general-head boundary with an upper threshold of flux into the aquifer. Flow can be into or out of the aquifer, but it can only inflow to an upper limit._

_The head-dependent flux boundaries can be used for other hydrologic conditions than their names suggest. Table 14.4 shows some other uses for these boundaries.
The head-dependent flux boundaries are similar in behavior to a constant-head boundary in an adjacent aquifer. In part e, a VCONT parameter of 200 $\frac{{ft^2}{d}}{(100 ft)^2} = \frac{O.O2}{d}$ was equivalent to a conductance and therefore gave similar answers as the general head boundary._

_The excellent comparison of the head-dependent flux boundaries to one another provides assurance that they are all implemented in the same fashion. The additional comparison to the two-layer model with adjacent constant head nodes provides a further check that they are implemented correctly in the model._

##### _Table 14.4. Other uses for the head-dependent flux boundary conditions in MODFLOW_

| General Head Boundary      | Drain                | River             | Evapotranspiration                  |
|----------------------------|----------------------|-------------------|-------------------------------------|
| Rivers                     | Intermittent streams | Adjacent aquifers | Drains with maximum flow limitation |
| Exterior model boundatires | Springs              |      --           | Wetlands                            |
| Adjacent aquifers          | Ditches              |    --             |      --                             |
